
# Debug hand pml in modesolver

In [1]:
import numpy as np

from fiberamp.fiber.microstruct.bragg import Bragg
from ngsolve.webgui import Draw
from ngsolve import CF, x, y, pml
import ngsolve as ng

importing NGSolve-6.2.2203-56-g47f483033


In [2]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ts = [15*2.7183333333333333e-6, 15*2/3*1e-6, 15*1e-6]
ns = [lambda x: n_air, lambda x: n_glass, lambda x: n_glass]
mats = ['air', 'glass', 'Outer']
maxhs = [.1, .04, .06]
scale = 15e-6
i=0
alpha=1
wls = np.linspace(1.4, 2, 301) * 1e-6

A = Bragg(ts=ts, scale=scale, maxhs=maxhs, ns=ns, mats=mats, wl=wls[i])


ModeSolver: Checking if mesh has required regions
Mesh has  3592  elements,  1849  points,  and  5440  edges.


In [3]:
# Set complex trafo
d=0
rh = ng.sqrt(ng.x**2 + ng.y**2)  # radial variable
r = rh + 1 / (d + 1) * alpha * 1j * (rh - A.R) ** (d + 1)
drdrh = 1 + alpha * 1j * (rh - A.R) ** d

x = r * ng.x / rh
y = r * ng.y / rh

trafo = ng.CoefficientFunction([(ng.x, ng.y), (ng.x, ng.y), (x, y)])

# derivatives and determinant

dxdxh = drdrh * (ng.x / rh) ** 2 + r * (ng.y ** 2 / rh ** 3)
dxdyh = drdrh * (ng.y * ng. x / rh ** 2) - \
    r * (ng.y * ng.x / rh ** 3)

dydyh = drdrh * (ng.y / rh) ** 2 + r * (ng.x ** 2 / rh ** 3)
dydxh = dxdyh

J = CF((dxdxh, dxdyh, dydxh, dydyh),
                           dims=(2, 2))

M = ng.CF((1+0*1j, 0+0*1j, 0+0*1j, 1+0*1j), dims=(2,2))

Jf = CF([M, M, J])

custom_pml = pml.Custom(trafo, Jf)

In [4]:
custom_pml.PML_CF

In [16]:
trafo = ng.CF([(ng.x, ng.y) if i != len(A.ns)-1 else (x, y) for i in range(len(A.ns))])


In [14]:
cf = CF(L)

In [15]:
Draw(cf, A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [11]:
det, JinvT, trafo = A.set_smooth_pml(1,0)

Set handmade custom (smooth) PML with alpha= 1 , and smoothing factor = 0


In [12]:
radial = pml.Radial(rad=A.R, alpha=alpha*1j, origin=(0, 0))


In [20]:
det2 = radial.Det_CF
Jinvng = radial.JacInv_CF

In [40]:
val = det(A.mesh(A.R))

In [41]:
det = CF([val, val, det])

In [43]:
Draw(det,A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene